## MONET-Analysis AERONET prep notebook

### How to use

- start notebook and 
- in cell 2 set the start date and end date
- in cell 2 set the filename output (something like AERONET_L15_STARTDATE_ENDDATE.nc with STARTDATE and ENDDATE in YYYYMMDD format)

In [1]:
import monetio as mio
import numpy as np
import pandas as pd
import xarray as xr
from util import write_util
import os

In [2]:
# set the dates
### for some reason the 7th of august 2019 has an error reading in the parallel version.  
# Not sure why but some of the dtypes are different when processing in parallel vs serial.  
# Here we will use the serial version just to make it easy for everyone to understand.        
### For faster processing in other periods add the `n_procs=n` kwarg where n is the number of processes
dates = pd.date_range(start='2019-09-01',end='2019-09-30',freq='H')
# set the output filename
outname = 'AERONET_L15_20190901_20190930.nc'
# set standard wavelengths
standard_wavelengths = np.array([0.34, 0.44, 0.55, 0.66, 0.86, 1.63, 11.1])* 1000. # convert from micron to nm 
# get the data 
df = mio.aeronet.add_data(dates, interp_to_aod_values=standard_wavelengths, freq='H') # ,n_procs=12)

Reading Aeronet Data...


In [6]:
# dfp = df.rename({'siteid':'x'},axis=1).set_index(['time','x']).drop_duplicates()
# df = df3 
verbose=False
dfp = df.rename({'siteid':'x'},axis=1).set_index(['time','x'])
columns = dfp.columns.to_list()
columns2 = []
remove_columns = []
for i in np.arange(len(columns)):
    columns2.append(columns[i])
    try:
        dfp[columns2].to_xarray()
        if verbose:
            print('COLUMN SUCCESS:',columns[i])
    except:
        if verbose:
            print('COLUMN FAILURE:',columns[i])
        remove_columns.append(columns[i])
        columns2.remove(columns[i])
if verbose:
    print(columns2)
dft = df.drop(remove_columns,axis=1)
dfp = dfp.drop(remove_columns,axis=1).dropna(subset=['latitude','longitude'])
dfx = dfp.to_xarray()

In [7]:
dsets = []
for s in df.siteid.unique():
    dsets.append(dft.loc[df.siteid == s].set_index(['time']).to_xarray())

In [8]:
# now site_variable are the single element attributes of the individual site so lets simplify this

from numpy import unique
site_variable = ['siteid','latitude','longitude','aeronet_instrument_number','elevation']

def expand_dims(ds, index=0):
    # first set a new index for the siteid
    ds['x'] = index
    ds = ds.expand_dims(['x'])
    ds = ds.set_coords(['x'])
    # now reduce the site variables to single element variables 
    for sv in site_variable:
        tmp = [unique(ds[sv])[0]]
        ds[sv] = (('x',), tmp)
    return ds

In [9]:
# now site_variable are the single element attributes of the individual site so lets simplify this
for index,d in enumerate(dsets):
    dsets[index] = expand_dims(d,index=index)

In [10]:
# now combine all the datasets for each site into a single dataset
ds = xr.concat(dsets,dim='x').set_coords(site_variable)
# ds
os.path.join

<function posixpath.join(a, *p)>

In [12]:
# write the file 
t = ds.expand_dims('y').transpose('time','y','x')
write_util.write_ncf(t,os.path.join('data',outname))

Writing: data/AERONET_L15_20190901_20190930.nc
Compressing: day_of_year, original_dtype: float64
Compressing: day_of_year(fraction), original_dtype: float64
Compressing: aod_1640nm, original_dtype: float64
Compressing: aod_1020nm, original_dtype: float64
Compressing: aod_870nm, original_dtype: float64
Compressing: aod_865nm, original_dtype: float64
Compressing: aod_779nm, original_dtype: float64
Compressing: aod_675nm, original_dtype: float64
Compressing: aod_667nm, original_dtype: float64
Compressing: aod_620nm, original_dtype: float64
Compressing: aod_560nm, original_dtype: float64
Compressing: aod_555nm, original_dtype: float64
Compressing: aod_551nm, original_dtype: float64
Compressing: aod_532nm, original_dtype: float64
Compressing: aod_510nm, original_dtype: float64
Compressing: aod_500nm, original_dtype: float64
Compressing: aod_490nm, original_dtype: float64
Compressing: aod_443nm, original_dtype: float64
Compressing: aod_412nm, original_dtype: float64
Compressing: aod_400nm, o

In [13]:
ls -lh data/AERONET*

-rw-r--r-- 1 bbaker25 users 16M May 18 13:56 data/AERONET_L15_20190801_20190831.nc
-rw-r--r-- 1 bbaker25 users 14M Jun  7 11:19 data/AERONET_L15_20190901_20190930.nc
